## Sistemas de Recomendación como en clase adaptado con Typo Producto

En este lab vamos a emplear la librería surprise. Esta es una librería que se basa en la API de scikit-learn y permite implementar varios algoritmos básicos de recomendación.



In [1]:
from surprise import SVD
from surprise.model_selection import GridSearchCV

Probamos con los datos de Pippo

In [2]:
import pandas as pd

In [3]:
#Usaremos los datos de los clientes que repiten compras a lo largo del año analizado
data = pd.read_csv('DataTypo_reducido.csv')
#Elimino la columna Unnamed: 0
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
data.shape

(246858, 3)

In [5]:
data.sample(5)

,customerId,productId,purchase_count
161168,71084,Toallas y Toallones,1
1657,491,Toallas y Toallones,2
156765,68349,Acolchados y Frazadas,1
171144,78040,Almohadones,2
50001,16744,Acolchados y Frazadas,1


In [6]:
mlens = data.rename(columns={"customerId": 'user_id', "productId" : 'item_id', "purchase_count": "rating"})

In [7]:
mlens.head(5)

,user_id,item_id,rating
0,0,Toallas y Toallones,1
1,0,Toallas y Toallones,1
2,0,Sábanas,1
3,0,Sábanas,1
4,0,Sábanas,1


El paquete surprise no recibe directamente un objeto DataFrame sino que tiene para parsear y leer un conjunto de datos debe hacerlo a través de dos nuevos objetos: Reader y Dataset. En Reader debemos especificar el valor mínimo y el valor máximo de los ratings y Dataset nos permite leer datos desde distintas fuentes.

In [8]:
lower_rating = mlens["rating"].min() 
upper_rating = mlens["rating"].max()
print('Review range: {0} to {1} '.format(lower_rating, upper_rating)) 

Review range: 1 to 5 


In [9]:
from surprise import Dataset, Reader
reader = Reader(rating_scale=(mlens["rating"].min(),mlens["rating"].max()))

In [10]:
dataset = Dataset.load_from_df(mlens,reader)

In [11]:
dataset

Ahora cargue SVD y GridSearchCV, ambos de surprise. Nota: GridSearchCV no está en surprise.GridSearchCV, surprise.GridSearch está deprecado.

In [12]:
from surprise import SVD
from surprise.model_selection import GridSearchCV

Genere una grilla de parámetros donde se prueben distintas combinaciones de:
    - epochs: es la cantidad de pasadas sobre el dataset que hará el algoritmo empleando descenso por el gradiente
    - biased: usar parámetros de sesgo o no
    - lr_all: learning rate para todos los parámetros
    - reg_all: término de regularización para todos los parámetros (lambda)

In [13]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}

Emplee GridSearchCV, SVD y el diccionario con los parámetros para probar, y entrene un modelo. Note que a GridSearchCV necesita pasarle un modelo sin instanciar. Además, setee el parámetro refit a True y con measures = ["rmse","fcp"]

In [14]:
gs = GridSearchCV(SVD, param_grid, measures=['fcp',"rmse"], cv=3, refit=True)

In [15]:
gs.fit(dataset)

Imprima el rmse y el fcp, y la mejor combinación de parámetros

In [16]:
gs.best_score

{'fcp': 0.5473808283049649, 'rmse': 0.4624696065047107}

In [17]:
gs.best_params

{'fcp': {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6},
 'rmse': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}}

Guarde el modelo con mayor fcp y prediga el rating para el user id 2 e item id "Toallas y Toallones"

In [18]:
best_model = gs.best_estimator["fcp"]

In [19]:
best_model.predict("2", "Toallas y Toallones")

Prediction(uid='2', iid='Toallas y Toallones', r_ui=None, est=1.190428638965168, details={'was_impossible': False})

Pruebe empleando otros modelos como SVDpp, NMF, KNNWithZScore e intente superar el valor obtenido

In [20]:
from surprise import SVDpp
gs = GridSearchCV(SVDpp, param_grid, measures=['fcp',"rmse"], cv=3, refit=True)
gs.fit(dataset)
gs.best_score

{'fcp': 0.5468265078489053, 'rmse': 0.46283629903633744}